# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [12]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [13]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [14]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [15]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [16]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [17]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [18]:
import numpy as np
from numpy.testing import assert_allclose

# Generate a random diagonally dominant matrix for testing
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# Jacobi iteration
def jacobi_iteration(A, b, n_iter):
    diag_1d = np.diag(A)
    
    B = -A.copy()
    np.fill_diagonal(B, 0)
    
    D = np.diag(diag_1d)
    invD = np.diag(1./diag_1d)
    BB = invD @ B
    c = invD @ b

    # Initial guess
    x = np.ones_like(b)
    
    for _ in range(n_iter):
        x = BB @ x + c
    
    return x

# Testing Jacobi iteration
n_iter = 50
x_jacobi = jacobi_iteration(A, b, n_iter)

# Check the result
print("Jacobi Iteration Result:", A @ x_jacobi - b)

# Task I.1: Check convergence with smaller diagonal elements
A_smaller_diag = rndm.uniform(size=(n, n)) + np.diagflat([5]*n)
x_jacobi_smaller_diag = jacobi_iteration(A_smaller_diag, b, n_iter)
BB_smaller_diag = np.diag(1./np.diag(A_smaller_diag)) @ (-A_smaller_diag + np.diag(np.diag(A_smaller_diag)))

print("Norm of B for smaller diagonal elements:", np.linalg.norm(BB_smaller_diag))


Jacobi Iteration Result: [ 1.11022302e-16  0.00000000e+00 -2.22044605e-16 -1.11022302e-16
  1.11022302e-16  0.00000000e+00 -2.42861287e-17  0.00000000e+00
 -2.77555756e-17  1.11022302e-16]
Norm of B for smaller diagonal elements: 1.0241070526789309


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [19]:
# Seidel's iteration (Gauss-Seidel method)
def seidel_iteration(A, b, n_iter):
    L = np.tril(A)
    U = A - L
    x = np.ones_like(b)
    
    for _ in range(n_iter):
        x = np.linalg.inv(L) @ (b - U @ x)
    
    return x

# Testing Seidel's iteration
x_seidel = seidel_iteration(A, b, n_iter)

# Check the result
print("Seidel Iteration Result:", A @ x_seidel - b)


Seidel Iteration Result: [ 0.00000000e+00 -1.11022302e-16 -2.22044605e-16 -1.11022302e-16
  1.11022302e-16  0.00000000e+00 -3.46944695e-18  0.00000000e+00
  2.77555756e-17  0.00000000e+00]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [20]:
# Minimum residual scheme
def minimum_residual(A, b, n_iter):
    x = np.ones_like(b)
    r = b - A @ x
    
    for _ in range(n_iter):
        Ar = A @ r
        tau = (r @ Ar) / (Ar @ Ar)
        x = x + tau * r
        r = b - A @ x
    
    return x

# Testing minimum residual scheme
x_minres = minimum_residual(A, b, n_iter)

# Check the result
print("Minimum Residual Scheme Result:", A @ x_minres - b)


Minimum Residual Scheme Result: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.46944695e-18 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
